In [1]:
# Imports
import numpy as np
import pandas as pd
import re
import csv
import random
import math
import sys

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Bidirectional
from keras.optimizers import SGD, rmsprop

Using TensorFlow backend.


In [2]:
#turns a sequence into a question format
# seq - whole dataset interger sequence
# l   - length of the question sequence
# c   - number of answer choices available
# p   - probability of missing element being last in the sequence
def toQuestion(seq,l,c,p):
    i = {}
    l = min(len(seq), l)
    
    #question
    a = random.randint(0,len(seq)-l)
    b = a+l-1
    s = seq[a:b]
    sd = math.ceil(np.std(s))
    if random.random() < p:
        m = len(s)-1              #use last element as missing
    else:
        m = random.randint(0,len(s)-1)   #random missing element
    
    #add sequence
    ans = s[m]
    s[m] = "?"    #set as missing
    i["sequence"] = s
    #i["sequence"] = ",".join([str(j) for j in s])
    
    #make randomly generated choices
    cs = []
    cs.append(ans)   #add right answer
    
    while len(cs) < c:       #add fake answers
        e = math.floor(ans+sd*random.uniform(-2,3))    #vary the answer choices using std and uniform randomness
        if e not in cs:
            cs.append(e)
    
    random.shuffle(cs)       #shuffle order
    i["options"] = cs
    
    #set answer to the missing element and add
    i["answer"] = np.where(np.array(cs)==ans)[0][0]
    
    return i



In [3]:
a = list(csv.reader(open('./synthetic sequences2.csv')))

In [4]:
for item in a:
    item[:] = [x for x in item if x != '']

In [5]:
for item in a:
    item[:] = list(map(lambda x: float(x), item))

In [6]:
seq_class1 = []
for item in a:
    #print (item)
    seq_class1 = seq_class1 + [toQuestion(item, random.randint(5, len(item)), 4, 0.75)['sequence']]

In [7]:
class1 = [1] * len(seq_class1)

In [8]:
#load and clean competition dataset
seqdataIn = pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.json', orient='records')
seqDataAns =pd.read_json('/Volumes/GoogleDrive/My Drive/NYU/PhD/20 DL/project/data/seq-public.answer.json',orient='index')

#clean hints
def hint2txt(h):
    return h.replace('<sub>','_{').replace('</sub>','}').replace('<sup>','^{').replace('</sup>','}').replace('\n',' | ').replace('\*','*')
seqDataAns['hint'] = seqDataAns['hint'].map(lambda x: hint2txt(x))

#split the hint by '=', to search item in second term (after '=')
seqDataAns['hint1'] = [item.split('=') for item in seqDataAns['hint']]

In [9]:
#instantiate the classes variable
seqDataAns['classes'] = 0

#assign the class for each sequence
#excludes sequences with error, which remain as class 0
for i in range(len(seqDataAns)):
    if i in [82,  257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628]:
        continue
        
    if seqDataAns['classes'][i] == 0:
        y = False
        z = False
        if len(seqDataAns['hint1'][i])>1:
            y = re.search('A_{', seqDataAns['hint1'][i][1])
            z = re.search('(?<!_).*?n', seqDataAns['hint1'][i][1])

        if (y) and not(z):
            seqDataAns.loc[i, 'classes'] = 1   #recursive A_{n}
            
        elif not(y) and (z):
            seqDataAns.loc[i, 'classes'] = 2   #just the position n
        
        elif not(y) and not(z):
            seqDataAns.loc[i, 'classes'] = 0   #none, or errors
            
        else:
            seqDataAns.loc[i, 'classes'] = 3   #both
            
        

In [10]:
#dataset distribution
seqDataAns.classes.value_counts()

3    511
2    380
0    184
1      1
Name: classes, dtype: int64

In [11]:
#use the sequences/classes to train the classifier
sequences = seqdataIn['stem']


In [12]:
#cleans up sequences
def txt2seq(seq):
    s2 = []
    for s in seq:
        si = re.sub("\(\s?\)","?",s)       #replace parenthesis with question mark
        a = re.split(',|\s|-',si)      #remove any delimiters
        a = [e for e in a if e != ""]  #remove empty strings
        s2.append(a)
    return s2
split_seq = txt2seq(sequences)

In [13]:
for item in seq_class1:
    item[:] = [str(item2) for item2 in item]

In [14]:
split_seq.extend(seq_class1)

In [15]:
classes = seqDataAns.classes

In [16]:
classes = classes.append(pd.Series(class1))
classes = classes.reset_index(drop=True)

In [17]:
classes.value_counts()

3    511
2    380
1    185
0    184
dtype: int64

In [18]:
#checks for valid sequences (all numbers or fractions excluding ?)
def findBadSeq(seq):
    badSeqs = {}
    for a in range(len(seq)):     #check each sequence
        s = seq[a]
        for i in s:   #check each item (assuming sequence is already split)
            if(re.match(r'[^0-9\/\?\-]',i)):    #check if any words or weird characters in the sequence
                badSeqs[a] = s
                break
    return badSeqs

#remove bad sequences from good sequences
def remBadSeq(seq):
    badSeqInd = list(findBadSeq(seq).keys())
    goodSeq = {}
    for i in range(len(seq)):
        if i not in badSeqInd:     #want to keep the indexes to get the answer for it
            goodSeq[i] = seq[i]
    return goodSeq

In [19]:
# show bad sequences removed
bs = findBadSeq(split_seq)
for k,v in bs.items():
    print(str(k) + ":" + str(v))
print("# Bad sequences = " + str(len(bs)) + "/" + str(len(split_seq)))

24:['√2', '3', '√28', '√65', '?']
85:['A', 'F', 'H', 'K', 'N', '?', '?']
86:['A', 'D', 'I', 'P', 'Y', 'CF', 'DI', 'FD', '?']
118:['+1', '1', '1', '1', '?']
161:['19/13', '1', '19/13', '10/22', '?', 'a.7/24;b.7/25;c.5/26;d.7/26;']
297:['√5', '√55', '11√5', '11√55', '?']
367:['0', '3', '1', '6', '√2', '12', '?', '?', '2', '48']
389:['1/2', '√3/2', '√3/3', '?']
411:['(√5', '1)/4', '1/3', '(√3', '1)/2', '√2', '1', '?']
470:['A', 'C', 'F', 'J', 'O', '?', 'What', 'letter', 'comes', 'next?']
487:['72', '(68)', '41', '28', '(98)', '16', '34', '?', '56']
489:['4342', '(3176)', '1726', '7995', '(7516)', '2162', '8418', '?', '1725']
491:['3', '+6', '+2', '3', '+7', '12', '?']
608:['961', '(25)', '432', '932', '?', '731']
609:['16', '(96)', '12', '10', '?', '7.5']
610:['The', 'following', 'series', 'of', 'numbers', 'contains', 'one', 'number', 'that', 'does', 'not', 'fit', 'the', 'pattern', 'set', 'by', 'the', 'others.', 'What', 'number', 'does', 'not', 'fit?', '3', '5', '7', '11', '14', '17']
615

In [20]:
#get only good sequences
gs = remBadSeq(split_seq)
for k,v in gs.items():
    print(str(k) + ":" + str(v))
print("# Good sequences = " + str(len(gs)) + "/" + str(len(split_seq)))

0:['7', '9', '1', '5', '?']
1:['3', '2', '5/3', '3/2', '?']
2:['1', '2', '5', '26', '?']
3:['2', '12', '30', '?']
4:['2', '1', '2/3', '1/2', '?']
5:['4', '2', '2', '3', '6', '?']
6:['1', '7', '8', '57', '?']
7:['4', '12', '8', '10', '?']
8:['1/2', '1', '1', '?', '9/11', '11/13']
9:['95', '88', '71', '61', '50', '?']
10:['2', '6', '13', '39', '15', '45', '23', '?']
11:['1', '3', '3', '5', '7', '9', '13', '15', '?', '?']
12:['1', '2', '8', '28', '?']
13:['0', '4', '18', '?', '100']
14:['1', '1', '2', '2', '3', '4', '3', '5', '?']
15:['1', '52', '313', '174', '?']
16:['5', '15', '10', '215', '?']
17:['7', '0', '1', '2', '9', '?']
18:['0', '1', '3', '10', '?']
19:['9/2', '14', '65/2', '?', '217/2']
20:['1', '1', '2', '6', '24', '?']
21:['3', '4', '8', '24', '88', '?']
22:['20', '22', '25', '30', '37', '?']
23:['1/9', '2/27', '1/27']
25:['1', '2', '4', '8', '16', '?']
26:['2', '1', '2/3', '1/2', '?']
27:['1', '1', '3', '7', '17', '41', '?']
28:['5/2', '5', '25/2', '75/2', '?']
29:['6', '15'

In [21]:
#converts all sequence items to float values
def floatConv(s):
    try:
        return float(s)
    except ValueError:
        try:
            num, denom = s.split('/')
            return float(num) / float(denom)
        except ValueError:
            return 'X'   #bad

#converts all the sequences to floats
def seq2Float(seq):
    floatSeq = {}
    for i in seq.keys():
        s = list(map(lambda x: floatConv(x) if x != "?" else x, seq[i]))
        if "X" in s:  #found a bad one (don't use)
            continue
        else:
            floatSeq[i] = s
    return floatSeq
print(seq2Float(gs))

{0: [7.0, 9.0, 1.0, 5.0, '?'], 1: [3.0, 2.0, 1.6666666666666667, 1.5, '?'], 2: [1.0, 2.0, 5.0, 26.0, '?'], 3: [2.0, 12.0, 30.0, '?'], 4: [2.0, 1.0, 0.6666666666666666, 0.5, '?'], 5: [4.0, 2.0, 2.0, 3.0, 6.0, '?'], 6: [1.0, 7.0, 8.0, 57.0, '?'], 7: [4.0, 12.0, 8.0, 10.0, '?'], 8: [0.5, 1.0, 1.0, '?', 0.8181818181818182, 0.8461538461538461], 9: [95.0, 88.0, 71.0, 61.0, 50.0, '?'], 10: [2.0, 6.0, 13.0, 39.0, 15.0, 45.0, 23.0, '?'], 11: [1.0, 3.0, 3.0, 5.0, 7.0, 9.0, 13.0, 15.0, '?', '?'], 12: [1.0, 2.0, 8.0, 28.0, '?'], 13: [0.0, 4.0, 18.0, '?', 100.0], 14: [1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 3.0, 5.0, '?'], 15: [1.0, 52.0, 313.0, 174.0, '?'], 16: [5.0, 15.0, 10.0, 215.0, '?'], 17: [7.0, 0.0, 1.0, 2.0, 9.0, '?'], 18: [0.0, 1.0, 3.0, 10.0, '?'], 19: [4.5, 14.0, 32.5, '?', 108.5], 20: [1.0, 1.0, 2.0, 6.0, 24.0, '?'], 21: [3.0, 4.0, 8.0, 24.0, 88.0, '?'], 22: [20.0, 22.0, 25.0, 30.0, 37.0, '?'], 23: [0.1111111111111111, 0.07407407407407407, 0.037037037037037035], 25: [1.0, 2.0, 4.0, 8.0, 16.0, '?

In [22]:
#prepare sequences for classifier
valid = seq2Float(gs)  #good sequences dictionary
len(valid)

1217

In [23]:
maxx = 0
for key in valid.keys():
    if len(valid.get(key))>maxx:
        maxx = len(valid.get(key))
maxx

19

In [24]:
secuencias = []
clases = []
for key in valid.keys():
    if key in [82, 257, 298, 306, 451, 452, 463, 464,468, 476,495, 498,499, 500,504,505,623,624,626,627,628, 
               23, 501, 639, 697, 1056, 1073,
               11, 134, 158, 253, 467, 470, 475, 636, 769, 793, 816, 818, 833, 856, 862, 945]:
        continue


    secuencias.append(valid.get(key))
    clases.append(classes[key])

In [25]:
question = [] #position of question mark
seq_len = []
for i in range(len(secuencias)):
    question.append(secuencias[i].index('?'))
    seq_len.append(len(secuencias[i]))
    




In [26]:
secuencias2 = secuencias[:] #replace '?' by zero
for i in range(len(secuencias2)):
    secuencias2[i][question[i]] = 0

In [27]:
secuencias3 = pad_sequences(secuencias2, padding='post') #make all sequences same lenght adding zeros

In [28]:
secuencias3 = secuencias3.tolist()

In [29]:
mask = []
for i in range(len(secuencias3)):
    mask.append([0] * len(secuencias3[0])) 

In [30]:
for i in range(len(mask)):
    for j in range(seq_len[i]):
        mask[i][j] = 1  #assign 1 in the mask sequence for actual values in sequence

In [31]:
for i in range(len(mask)):
    mask[i][question[i]] = 2 #assign 2 in the mask sequence for question mark position

In [32]:
#concatenate the sequence and the mask
secuencias4 = []
for i in range(len(secuencias3)):
    secuencias4.append(secuencias3[i]+mask[i])

In [33]:
#Classifier
#change question mark with '-1' to use the single serie
secuencias5 = secuencias3[:] 
for i in range(len(secuencias5)):
    secuencias3[i][question[i]] = -1





In [34]:
X_train, X_test, y_train, y_test = train_test_split(secuencias4, clases, test_size=0.15, stratify = clases)#, random_state=42) #train using the mask
#X_train, X_test, y_train, y_test = train_test_split(secuencias5, clases, test_size=0.2, random_state=42) #train with just the serie




In [35]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [36]:
X_train = X_train.reshape((len(X_train), len(X_test[0]), 1)) 
X_test = X_test.reshape((len(X_test), len(X_test[0]),1))


y_test = pd.get_dummies(y_test).values
y_train = pd.get_dummies(y_train).values




y_train = y_train.reshape((len(y_train), len(np.unique(clases)), 1))
y_test = y_test.reshape((len(y_test), len(np.unique(clases)), 1))

In [37]:
y_test.shape

(177, 4)

In [44]:
model = Sequential()
#model.add(LSTM(100, input_shape= (19, 1)))
model.add(LSTM(100, input_shape= (38, 1)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, batch_size = 8)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                3232      
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 132       
Total params: 44,164
Trainable params: 44,164
Non-trainable params: 0
_________________________________________________________________
None
Train on 1000 samples, validate on 177 samples
Epoch 1/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4893 - accuracy: 0.7640 - val_loss: 0.4349 - val_accuracy: 0.7754
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4450 - accuracy: 0.7782 - val_loss: 0.4408 - val_accuracy: 0.7627
Epoch 3/100
1000/1000 [===============

1000/1000 [==============================] - 3s 3ms/step - loss: 0.3466 - accuracy: 0.8418 - val_loss: 0.4380 - val_accuracy: 0.7994
Epoch 52/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3432 - accuracy: 0.8440 - val_loss: 0.4160 - val_accuracy: 0.8249
Epoch 53/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3483 - accuracy: 0.8485 - val_loss: 0.3876 - val_accuracy: 0.8079
Epoch 54/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3481 - accuracy: 0.8445 - val_loss: 0.4109 - val_accuracy: 0.8136
Epoch 55/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3478 - accuracy: 0.8447 - val_loss: 0.3881 - val_accuracy: 0.8037
Epoch 56/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3404 - accuracy: 0.8470 - val_loss: 0.4148 - val_accuracy: 0.8051
Epoch 57/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3426 - accuracy: 0.8470 - val_loss: 0.4029 - val_accuracy

In [ ]:
model.save("classifier")
#reconstructed_model = keras.models.load_model("classifier")